# Chocolate Rating Analysis

## Web scraping data to then process, clean, etc. in the notebook.

In [1]:
# import warnings
# warnings.filterwarnings('ignore')

# from requests import get
# from bs4 import BeautifulSoup
# import pandas as pd
# from selenium import webdriver
# import chromedriver_autoinstaller
# def create_driver(headless=True):
#     driver = chromedriver_autoinstaller.install(cwd=True)
#     chrome_options = webdriver.ChromeOptions()     
#     if headless:
#         chrome_options.add_argument("--headless")
#     driver = webdriver.Chrome(driver, 
#                              chrome_options = chrome_options)
#     return driver
# driver = create_driver()

In [2]:
# driver.get("http://flavorsofcacao.com/chocolate_database.html")

In [3]:
# div_id = "spryregion1"

# html = driver.page_source
# soup = BeautifulSoup(html)
# div = soup.find('div', {'id':div_id})
# # div

# table = div.find('table')
# table


In [4]:
# pd.read_html(str(table))[0]

In [5]:
# df.to_csv('../../../data/chocolate.csv', index=False)

## Importing Necessary Python Libraries 

In [27]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import plot_confusion_matrix, auc, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from imblearn.over_sampling import SMOTE

all_data = pd.read_csv('../../../data/chocolate.csv')
all_data.head()

,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76%,"3- B,S,C","cocoa, blackberry, full body",3.75
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76%,"3- B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76%,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,797,A. Morin,France,2012,Peru,Peru,63%,"4- B,S,C,L","fruity, melon, roasty",3.75
4,797,A. Morin,France,2012,Bolivia,Bolivia,70%,"4- B,S,C,L","vegetal, nutty",3.50


## Data Cleaning 

### Replacing column spaces with underscores and lower casing column names for simplicity when calling them. 

In [28]:
#breaking away from original dataset and creating a copy to maintain the original structure as "all_data"
df = all_data.copy()

In [29]:
#removing parenthesis and spaces from columns 
df.columns = df.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.lower()

#removing percent signs from row cocoa_percent
df = df.replace("%", "", regex=True)

df.head()

,ref,company_manufacturer,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,ingredients,most_memorable_characteristics,rating
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76,"3- B,S,C","cocoa, blackberry, full body",3.75
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76,"3- B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,797,A. Morin,France,2012,Peru,Peru,63,"4- B,S,C,L","fruity, melon, roasty",3.75
4,797,A. Morin,France,2012,Bolivia,Bolivia,70,"4- B,S,C,L","vegetal, nutty",3.50


### Removing the unnecissary information leading up to the ingredient names

In [30]:
# retreiving rows from the third index and on - leaving out rows with empty values
df['ingredient_list'] = [np.nan if l is np.nan else l[3:] for l in df['ingredients']]

### Inspecting the value (occurance count) and leaving out those with a low number of occurances

In [31]:
df['ingredient_list'].value_counts()

B,S,C          903
B,S            661
B,S,C,L        273
B,S,C,V,L      184
B,S,C,V        139
B,S*            31
B,S*,C,Sa       20
B,S*,C          12
B,S,L            8
B,S*,C,V         7
,S,C,V,Sa        6
B,S,C,Sa         5
B,S,V,L          5
B                5
,S,C,V,L,Sa      4
B,S,V            3
B,S*,V,L         3
B,S*,C,L         2
B,S,C,L,Sa       1
B,S*,Sa          1
B,C              1
Name: ingredient_list, dtype: int64

In [32]:
# leaving out occurances below 8 

most_common = (df['ingredient_list'].value_counts()).iloc[:9].index


df = df[df['ingredient_list'].isin(most_common)]


In [33]:
df

,ref,company_manufacturer,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,ingredients,most_memorable_characteristics,rating,ingredient_list
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76,"3- B,S,C","cocoa, blackberry, full body",3.75,"B,S,C"
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76,"3- B,S,C","cocoa, vegetal, savory",3.50,"B,S,C"
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76,"3- B,S,C","rich cocoa, fatty, bready",3.25,"B,S,C"
3,797,A. Morin,France,2012,Peru,Peru,63,"4- B,S,C,L","fruity, melon, roasty",3.75,"B,S,C,L"
4,797,A. Morin,France,2012,Bolivia,Bolivia,70,"4- B,S,C,L","vegetal, nutty",3.50,"B,S,C,L"
...,...,...,...,...,...,...,...,...,...,...,...
2357,1205,Zotter,Austria,2014,Blend,Raw,80,"4- B,S*,C,Sa","waxy, cloying, vegetal",2.75,"B,S*,C,Sa"
2358,1996,Zotter,Austria,2017,Colombia,"APROCAFA, Acandi",75,"3- B,S,C","strong nutty, marshmallow",3.75,"B,S,C"
2359,2170,Zotter,Austria,2018,Belize,Maya Mtn,72,"3- B,S,C","muted, roasty, accessible",3.50,"B,S,C"
2360,2170,Zotter,Austria,2018,Congo,Mountains of the Moon,70,"3- B,S,C","fatty, mild nuts, mild fruit",3.25,"B,S,C"


In [34]:
df.ingredient_list.value_counts()

B,S,C        903
B,S          661
B,S,C,L      273
B,S,C,V,L    184
B,S,C,V      139
B,S*          31
B,S*,C,Sa     20
B,S*,C        12
B,S,L          8
Name: ingredient_list, dtype: int64

------

### -- Only use if we BREAK UP INGREDIENTS and MOST_MEM_CHAR --  Separating multiple into their own individual rows ----

In [13]:
#placing each ingredient into a new list sepparated by commas

# df.most_memorable_characteristics = df.most_memorable_characteristics.str.split(',')


# df['split_ingredients'] = df['ingredient_list'].str.split(',')

# df = df.explode('split_ingredients')
# df = df.explode('most_memorable_characteristics')


------------

------------

### Clarifying important predictor values (x) for the target value (y).

In [23]:
# dropping the target (y) values from x and specific origin
features = df.drop(['rating','specific_bean_origin_or_bar_name', 'ingredients'], axis =1)
X = features
y = df.rating


### Splitting the data into three subsets of training and validation data for the future models.
    Two train test splits create three subsets of the original dataset which allows for the training data to not be bled into the test data - this reduced model's bias towards the pre-existing testing data, thus assuring maximum performance on future test sets which the model has never been exposed. 

In [24]:
#performing train test split for test set (subsets 1/3)
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=.15, random_state=42)


#performing a train test split for train and validation set (subsets - 3/3)
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, test_size=.15, random_state=42) 

In [26]:
#checking to ensure the shape of the columns and rows are still the same for the X and y values after the train test split.
X_tr.shape, y_tr.shape

((2007, 8), (2007,))

### Replacing any existing missing values  

In [27]:
#ingredients have 88 rows that have no imputs.
X_val.isna().sum()

ref                                0
company_manufacturer               0
company_location                   0
review_date                        0
country_of_bean_origin             0
cocoa_percent                      0
most_memorable_characteristics     0
ingredient_list                   10
dtype: int64

In [18]:
# the empty value replacement will be done using the most frequent fill strategy
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns = X.columns)

X_val_imputed = pd.DataFrame(imputer.fit_transform(X_val),columns = X.columns)

In [19]:
#ensuring that the missing values from ingredients are now filled in with the most frequent occuring value

X_val_imputed.isna().sum()

ref                               0
company_manufacturer              0
company_location                  0
review_date                       0
country_of_bean_origin            0
cocoa_percent                     0
ingredients                       0
most_memorable_characteristics    0
ingredient_list                   0
dtype: int64

### Separating groups into numeric and catagorical data types 

In [20]:
#taking a look at the groups data types to ensure that they are separated correctly
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1705 entries, 108 to 624
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   ref                             1705 non-null   int64 
 1   company_manufacturer            1705 non-null   object
 2   company_location                1705 non-null   object
 3   review_date                     1705 non-null   int64 
 4   country_of_bean_origin          1705 non-null   object
 5   cocoa_percent                   1705 non-null   object
 6   ingredients                     1636 non-null   object
 7   most_memorable_characteristics  1705 non-null   object
 8   ingredient_list                 1636 non-null   object
dtypes: int64(2), object(7)
memory usage: 133.2+ KB


In [21]:
X_train_numeric 
X_train_cat

X_val_numerica

NameError: name 'X_train_numeric' is not defined

## Preprocessing:

In [ ]:
git fetch 

git merge/chandler